# Starcraft II build order prediction & classification

## Intro & Motivation

Interest in Starcraft II (SCII) in the AI/ML community has increased dramatically after the release of DeepMind's AlphaStar bot which was first demonstrated against a [professional player in Janurary 2019](https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii) then subsequently released onto the [public ladder for the Fall 2019 season](https://news.blizzard.com/en-us/starcraft2/22933138/deepmind-research-on-ladder).

One interesting observation made by professional casters and commentators is that the strategies employed by AlphaStar were static and inflexible in nature. In particular, the build order or sequence of structures and unit composition remained constrained to a small set of possibilities. *From examination of referenced games played by AlphaStar on the ladder as identified by the community* [(link)](https://www.reddit.com/r/MachineLearning/comments/d13yex/r_deepmind_starcraft_2_update_alphastar_is/). The most recent version of AlphaStar made known to the public is performing at the GrandMaster level (top 0.2% of players) but *fails to beat professional players consistently* (top ~100-200 players).

I take it that the training architecture of AlphaStar was able to refine the execution of particular strategies and exploit the predominant winning strategies employed during the current ladder season but fails to respond to novel strategies employed by human players in the wild. In essense, AlphaStar's *tactics* were impeccable but it lacks *strategy* discovery. To my limited understanding, the different strategies learnt by AlphaStar were mainly driven by *exploiter agents* in what appears to be an adversarial fashion [(DeepMind Blog, Figure 1)](https://deepmind.com/blog/article/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning).

One interesting development is the proposal of [*informed MCTS* (Santiago Ontanon, 2016)](https://ieeexplore.ieee.org/document/7860394). It is not clear whether AlphaStar utilized this approach as the author has since joined the Google Research team after the publishing of this paper. In traditional MCTS, the tree-policy is informed with UCB1 which does not make assumptions or attempt to exploit prior knowledge of the opponent's actions (Santiago, 2016). The bayesian approach introduced here is to inform exploration with a distribution of opponent actions.

## Project Scope

The framework setup for training agents in the SCII environment would be out-of-scope for this semester's small project. However, a crucial step towards eventually reaching the *informed MCTS* approach to SCII would be to learn a posterior distribution of opponent actions. **A possible first step in this direction would be to explore the different actions as a time-sequence that the player takes to make predictions at to what the player would do next. Furthermore, it would also be interesting to classify the existing strategies in an unsupervised manner to visualize patterns in the current ladder season.**

Therefore, the proposed scope of my semester project for CSE515T Bayesian ML is as follows in order:
1. Employ unsupervised classification on the existing build-orders with EM clustering.
    - Perhaps this could also serve to inform a prior distribution of possible actions
2. Employ Bayesian time-series prediction to SCII build-order dataset to predict possible distribution of future events.
    - I anticipate this task to be much harder and may be incomplete by the end of this semseter
    
As an immediate result of this research project, contributions may be made to the *spawningtool* python package directly:
> Currently, spawningtool offers a basic parser for extracting build orders from replays. Our goal is to incorporate more sophisticated techniques from Artificial Intelligence to understand and classify these build orders.

## Comment on being an 'expert' in the dataset

I was previously on our WashU E-Sports Starcraft II Team as a Diamond-League player (top 20% in 2019 season), admittedly I was not competing at the pro/semi-pro level. However, I've since followed with great interest both publications on AlphaStar as well as SCII tournaments over the past couple years. As such, I have good knowledge of popular build orders, their proper execution, and textbook responses to opponent strategies. In addition, I am aware of what to look out for when scouting enemy information and the approximate timings in matches where players are able to acquire such information.

Furthermore, I have been consistently involved in the [MIT Battlecode](https://battlecode.org/) competitions throughout the years since 2017 and thus have prior experience coding bots for playing RTS games. From my experience, a large portion of games are decided based on build-order wins rather than micro-intensive out-maneuvering between units. Therefore, the automation of build-order search and refinement strategies would be paramount to future success. To my knowledge, no such hyper-parameter tuning for discovering and countering build-orders has been employed in Battlecode to date and one hopes that work done here would help in next year's contest.

## Dataset

[lotv.spawningtool.com](lotv.spawningtool.com) has a vast collection of community uploaded replays and some basic labeling of general starting strategies as well as unit compositions (mixture of units to built) which are hand-labeled by the (human) community.

Furthermore, we have community tools for parsing such replays readily available such as [sc2reader](https://pypi.org/project/sc2reader/), [spawningtool](https://pypi.org/project/spawningtool/), and [Zephyrus Replay Parser](https://github.com/ZephyrBlu/zephyrus-sc2-parser). Each extracts information at different level of detail. I have preliminarily picked *spawningtool* for the broad sketch of what actions are taken by players within a match as demonstrated in the Short Demo section below.

2000 replays are extracted since the last major balance patch (where unit strengths and building costs are adjusted - therefore also potentially drastically changing strategies employed). From: https://lotv.spawningtool.com/zip/?pro_only=on&patch=150&query=&order_by=play&coop=n&after_played_on=8%2F14%2F20&before_played_on=3%2F14%2F21&before_time=15&after_time=&p=1. This will form the set of data for exploration in my project.

The same dataset can be extracted by running the bash script `./dl.sh` to queue a download of 80 zip folders of 25 replays each (2000 replays total).

# Short Demo

Below, I give a very brief exploration on one kind of data that we have access to in our dataset using the *spawningtool* parser.

In [1]:
import os
import sys
import spawningtool.parser

In [2]:
replay_dir = 'replays'
replays = []
for (dirpath, dirnames, filenames) in os.walk(replay_dir):
    replays.extend(filenames)
    break
print(replays)

['2021-03-04 - (P)Strange VS (Z)Lambo.SC2Replay', 'Clem v Zest Game 1 - Oxide LE.SC2Replay', 'Clem v Zest Game 2 - Lightshade LE.SC2Replay', 'Clem v Zest Game 4 - Deathaura LE.SC2Replay', 'Deathaura LE (11).SC2Replay', 'Deathaura_LE_250.SC2Replay', 'Deathaura_LE_70.SC2Replay', 'GabeTalks v Geralt - Pillars of Gold LE.SC2Replay', 'GabeTalks v MaxPax - Pillars of Gold LE.SC2Replay', 'Maru v Reynor Game 4 - Lightshade LE.SC2Replay', 'PartinG v TYTY Game 1 - Lightshade LE.SC2Replay', 'PartinG v TYTY Game 2 - Pillars of Gold LE.SC2Replay', 'PartinG v TYTY Game 3 - Romanticide LE.SC2Replay', 'PartinG v TYTY Game 4 - Deathaura LE.SC2Replay', 'PartinG v Zest Game 1 - Deathaura LE.SC2Replay', 'PartinG v Zest Game 2 - Pillars of Gold LE.SC2Replay', 'PartinG v Zest Game 3 - Romanticide LE.SC2Replay', 'PartinG v Zest Game 4 - Submarine LE.SC2Replay', 'PartinG v Zest Game 5 - Jagannatha LE.SC2Replay', 'Reynor v Zest Game 1 - Oxide LE.SC2Replay', 'Reynor v Zest Game 2 - Lightshade LE.SC2Replay', 'Re

In [3]:
sample_game = spawningtool.parser.parse_replay(os.path.join(replay_dir,replays[0]))

In [4]:
print(f"Parsed Replay Metadata labels: {list(sample_game.keys())}\n")
print(f"per-Player attributes accessible: {list(sample_game['players'][1].keys())}")

Parsed Replay Metadata labels: ['buildOrderExtracted', 'message', 'build', 'baseBuild', 'category', 'expansion', 'unix_timestamp', 'frames', 'game_type', 'region', 'map', 'map_hash', 'cooperative', 'players', 'include_map_details', 'frames_per_second']

per-Player attributes accessible: ['name', 'pick_race', 'race', 'league', 'level', 'is_winner', 'result', 'is_human', 'handicap', 'color', 'uid', 'region', 'supply', 'team', 'clock_position', 'commander', 'abilities', 'buildOrder', 'unitsLost']


In [8]:
# Simple function to extract first-n time and command issued by a player
def extract_buildOrder(game, player=1, limit=30):
    print(f"Player {player}: {game['players'][player]['race']}")
    return list(map(lambda x: f"{x['time']},{x['name']}", game['players'][player]['buildOrder']))[:limit]

Extraction of first 30 build commands issued by player 1 and timestamp of when the command was issued.

In [9]:
extract_buildOrder(sample_game, 1)

Player 1: Protoss


['0:00,Probe',
 '0:12,Probe',
 '0:20,Pylon',
 '0:24,Probe',
 '0:38,Probe',
 '0:39,Gateway',
 '0:41,Gateway',
 '0:47,Probe',
 '0:50,Assimilator',
 '0:53,Probe',
 '1:05,Probe',
 '1:17,Probe',
 '1:26,Nexus',
 '1:36,CyberneticsCore',
 '1:39,Probe',
 '1:45,Assimilator',
 '1:51,Probe',
 '1:54,Pylon',
 '2:03,Probe',
 '2:09,Adept',
 '2:15,Probe',
 '2:21,Stargate',
 '2:28,Probe',
 '2:29,Adept',
 '2:35,Warp Gate',
 '2:37,Probe',
 '2:45,Probe',
 '2:47,Probe',
 '2:52,Probe',
 '2:52,Probe']

Extraction of first 30 build commands issued by player 2.

In [10]:
extract_buildOrder(sample_game, 2)

Player 2: Zerg


['0:00,Drone',
 '0:13,Overlord',
 '0:17,Drone',
 '0:31,Drone',
 '0:31,Drone',
 '0:48,Hatchery',
 '0:52,Drone',
 '0:55,Drone',
 '0:58,Drone',
 '1:09,Extractor',
 '1:14,SpawningPool',
 '1:17,Drone',
 '1:20,Drone',
 '1:29,Drone',
 '1:39,Drone',
 '1:50,Drone',
 '2:01,Queen',
 '2:01,Queen',
 '2:01,Zergling',
 '2:01,Zergling',
 '2:02,Overlord',
 '2:11,Drone',
 '2:12,Drone',
 '2:13,zerglingmovementspeed',
 '2:22,Drone',
 '2:23,Drone',
 '2:36,Drone',
 '2:37,Drone',
 '2:40,Hatchery',
 '2:43,Zergling']